# Naive Bayes
* 优点：在数据较少的情况下仍然有效，可以处理多类别问题
* 缺点：对于输入数据的准备方式较为敏感
* 适用数据类型：标称型

### 准备数据：从文本中构建词向量

#### 词表到向量的转换
1 代表侮辱性文字， 0 代表正常

In [1]:
#词表到向量的转换函数
def loadDataSet():
    postingList = [['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'],
                 ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
                 ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'],
                 ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
                 ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'],
                 ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]
    classVec = [0,1,0,1,0,1]
    return postingList, classVec

def creatVocabList(dataset):
    vocabSet = set([])    #创建一个空集
    for document in dataset:
        vocabSet = vocabSet | set(document)  #创建两个集合的并集
    return list(vocabSet)

def setOfWords2Vec(vocabList, inputSet):
    returnVec = [0]*len(vocabList)  #创建一个其中所含元素都为0的向量
    for word in inputSet:   #遍历文档中所有单词
        if word in vocabList:
            returnVec[vocabList.index(word)] = 1  #如果出现了词汇表中的单词，则将输出的文档向量中的对应值设置为1
        else:
            print("the word: %s is no in my Vocabulary!" % word)
    return returnVec

### 训练算法：从词向量计算概率

In [2]:
from numpy import *
def trainNB0(trainMatrix, trainCategory):
    numTrainDocs = len(trainMatrix)
    numWords = len(trainMatrix[0])
    pAbusive = sum(trainCategory) / float(numTrainDocs)
    p0Num = ones(numWords) # p0Num = zeros(numWords)  #初始化概率
    p1Num = ones(numWords) # p1Num = zeros(numWords)
    p0Denom = 2.0 # p0Denom = 0.0
    p1Denom = 2.0 # p1Denom = 0.0
    for i in range(numTrainDocs):
        if trainCategory[i] == 1:
            p1Num += trainMatrix[i]   #向量相加
            p1Denom += sum(trainMatrix[i])  #向量相加
        else:
            p0Num += trainMatrix[i]
            p0Denom += sum(trainMatrix[i])
    p1Vect = log(p1Num/p1Denom) # p1Vect = p1Num / p1Denom   #对每个元素做除法
    p0Vect = log(p0Num / p0Denom) # p0Vect = p0Num / p0Denom
    return p0Vect, p1Vect, pAbusive

### 测试算法：根据实际情况修改分类器
* 利用贝叶斯分类器对文档进行分类时，要计算多个概率的乘积以获得文档属于某个类别的概率，如果其中一个概率值为0，那么最后的乘积也为0.为降低这种影响，可以将所有词出现数初始化为1，将分母初始化为2
* 太多很小的数相乘会造成下溢或者得不到正确答案，一种解决办法是对乘积取自然对数，来避免下溢出或者浮点数舍入导致的错误

#### NB分类函数

In [7]:
def classifyNB(vec2Classify, p0vec, p1vec, pClass1):
    p1 = sum(vec2Classify * p1vec) + log(pClass1)
    p0 = sum(vec2Classify * p0vec) + log(1.0 - pClass1)
    if p1 > p0:
        return 1
    else:
        return 0

### 测试

In [8]:
def testingNB():
    listOPosts, listClasses = loadDataSet()
    myVocabList = creatVocabList(listOPosts)
    trainMat = []
    for postinDoc in listOPosts:
        trainMat.append(setOfWords2Vec(myVocabList, postinDoc))
    p0v, p1v, pAb = trainNB0(array(trainMat),array(listClasses))
    testEntry = ['love', 'my', 'dalmation']
    thisDoc = array(setOfWords2Vec(myVocabList, testEntry))
    print(testEntry,"classified as:", classifyNB(thisDoc,p0v,p1v,pAb))
    testEntry = ['stupid','garbage']
    thisDoc = array(setOfWords2Vec(myVocabList, testEntry))
    print(testEntry, "classified as:", classifyNB(thisDoc, p0v, p1v, pAb))

In [9]:
testingNB()

['love', 'my', 'dalmation'] classified as: 0
['stupid', 'garbage'] classified as: 1


### 数据准备：文档词袋模型

In [10]:
#朴素贝叶斯词袋模型
def bagOfWords2VecMN(vocabList, inputSet):
    returnVec = [0]*len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] += 1
    return returnVec